In [ ]:
# Imports
import babypandas as bpd
import numpy as np

import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')

# Animation
from IPython.display import IFrame, display

def show_clt_slides():
    src = "https://docs.google.com/presentation/d/e/2PACX-1vTcJd3U1H1KoXqBFcWGKFUPjZbeW4oiNZZLCFY8jqvSDsl4L1rRTg7980nPs1TGCAecYKUZxH5MZIBh/embed?start=false&loop=false&delayms=3000"
    width = 960
    height = 509
    display(IFrame(src, width, height))

# Lecture 26 – Review
## DSC 10, Winter 2022

### Announcements

- The Final Project is due **tonight** ‼️
- Lab 8 is due **tomorrow at 11:59pm**.
- The **Final Exam** is on **Saturday 3/12 from 3-6PM**.
    - Administered remotely via Gradescope in the same format as Midterm Exam (multiple choice, short answer, fill-in-the-blank code).
    - Work on the practice exams posted on the [Resources tab of the course website](https://dsc10.com/resources/) and come to office hours with any questions.
    - See [this post](https://campuswire.com/c/G6950E967/feed/1403) on Campuswire for more details.
- If at least 85% of the class fills out **both** [CAPEs](https://cape.usd.edu) and the [additional survey](https://forms.gle/pt9HZ4RJzBrz1Vz98), then everyone will receive an extra 0.5% added to their overall course grade.
- Watch the review discussion [recording](https://youtu.be/FnsJ9xZPG2s), and look at the [statistics cheat sheet](https://campuspro-uploads.s3.us-west-2.amazonaws.com/6950e967-6500-4eae-8010-f961cccc4b93/a5fc93e3-9ece-4a48-9ba3-5653d571b5d8/dsc10%20stat%20cheat%20sheet.pdf).
- Look at the [Calendar](https://dsc10.com/calendar/) for the updated office hours schedule.
    - Come to the in-person Study Jam **tonight** from 3-7pm in the SDSC Auditorium!

### Agenda

- No new material – just review!

## The data: restaurants 🍟

Our data comes from [data.sfgov.org](https://data.sfgov.org/Health-and-Social-Services/Restaurant-Scores-LIVES-Standard/pyih-qa8i/data).

In [ ]:
restaurants = bpd.read_csv('data/restaurants_no_null.csv')
restaurants

It seems like the city for every row is `'San Francisco'`. We can confirm this using `np.unique`:

In [ ]:
np.unique(restaurants.get('business_city'))

As a result, the `'business_city'` column isn't telling us much. We won't look at many of the columns in our DataFrame, so let's just `get` the ones we're interested in.

In [ ]:
keep_cols = ['business_name', 'inspection_date', 'inspection_score', 'risk_category', 'Neighborhoods', 'Zip Codes']
restaurants = restaurants.get(keep_cols)
restaurants

## At-risk restaurants ⚠️

For each restaurant, we have an inspection score.

In [ ]:
restaurants

In the preview above, we see...
- A restaurant with an inspection score of 92 being classified as `'Low Risk'`,
- A restaurant with an inspection score of 91 being classified as `'High Risk'`
- A restaurant with an inspection score of 90 being classified as `'Low Risk'`

This means that inspection scores don't directly translate to risk categories. Let's investigate the difference between the inspection scores of low risk and high risk restaurants.

Let's start by visualizing the distribution of inspection scores for low risk and high risk restaurants.

In [ ]:
fig, ax = plt.subplots()
score_bins = np.arange(50, 102, 2)
restaurants[restaurants.get('risk_category') == 'Low Risk'].plot(
    kind='hist', y='inspection_score', density=True, ec='w', bins=score_bins, ax=ax,
    figsize=(10, 5), title='Inspection Scores for Low Risk vs. High Risk Restaurants', alpha=0.65, label='Low Risk'
);

restaurants[restaurants.get('risk_category') == 'High Risk'].plot(
    kind='hist', y='inspection_score', density=True, ec='w', bins=score_bins, ax=ax,
    figsize=(10, 5), alpha=0.65, label='High Risk'
);

### Discussion Question

We want to compare high risk restaurants to low risk restaurants and see if their inspection scores are significantly different. What technique should we use?

A. Standard hypothesis testing

B. Permutation testing  

C. Bootstrapping

D. The Central Limit Theorem

### To answer, go to [menti.com](https://menti.com) and enter the code 4857 5944.

<details>
<summary>Click for the answer <b>after</b> you've entered your guess above. <b>Don't scroll any further.</b></summary>
    
Permutation testing.

</details>

Let's keep only the relevant information.

In [ ]:
high_low = restaurants[(restaurants.get('risk_category') == 'Low Risk') | (restaurants.get('risk_category') == 'High Risk')]
high_low = high_low.get(['inspection_score', 'risk_category'])
high_low

Now, let's try shuffling a single one of the columns above. (Does it matter which one?)

In [ ]:
np.random.permutation(high_low.get('risk_category'))

Let's assign this shuffled column back into our original DataFrame. The resulting DataFrame is called `original_and_shuffled`.

In [ ]:
shuffled_labels = np.random.permutation(high_low.get('risk_category'))
original_and_shuffled = high_low.assign(shuffled_label=shuffled_labels)
original_and_shuffled

Let's now visualize the distribution of inspection scores for low risk and high risk restaurants, in both our original dataset and after shuffling the labels.

In [ ]:
fig, ax = plt.subplots()
score_bins = np.arange(50, 102, 2)
original_and_shuffled[original_and_shuffled.get('shuffled_label') == 'Low Risk'].plot(
    kind='hist', y='inspection_score', density=True, ec='w', bins=score_bins, ax=ax,
    figsize=(10, 5), title='Inspection Scores for Low Risk vs. High Risk Restaurants After Shuffling', alpha=0.65, label='Low Risk'
);

original_and_shuffled[original_and_shuffled.get('shuffled_label') == 'High Risk'].plot(
    kind='hist', y='inspection_score', density=True, ec='w', bins=score_bins, ax=ax,
    figsize=(10, 5), alpha=0.65, label='High Risk'
);

In [ ]:
fig, ax = plt.subplots()
score_bins = np.arange(50, 102, 2)
restaurants[restaurants.get('risk_category') == 'Low Risk'].plot(
    kind='hist', y='inspection_score', density=True, ec='w', bins=score_bins, ax=ax,
    figsize=(10, 5), title='Inspection Scores for Low Risk vs. High Risk Restaurants Before Shuffling', alpha=0.65, label='Low Risk'
);

restaurants[restaurants.get('risk_category') == 'High Risk'].plot(
    kind='hist', y='inspection_score', density=True, ec='w', bins=score_bins, ax=ax,
    figsize=(10, 5), alpha=0.65, label='High Risk'
);

### Discussion Question

It looks like the two groups in the second histogram are susbstantially more different than the two groups in the first histogram. 

What test statistic(s) can we use to quantify the difference between the two groups displayed in a given histogram?

A. Total Variation Distance  
B. Difference in group means  
C. Either of the above

### To answer, go to [menti.com](https://menti.com) and enter the code 4857 5944.

<details>
<summary>Click for the answer <b>after</b> you've entered your guess above. <b>Don't scroll any further.</b></summary>
    
Difference in group means. TVD helps compare two categorical distributions, but we're dealing with two numerical distributions.

</details>

In [ ]:
original_and_shuffled.groupby('risk_category').mean()

Let's compute the difference in mean inspection scores for the low risk group and high risk group (low minus high).

First, for our observed data:

In [ ]:
grouped = original_and_shuffled.groupby('risk_category').mean()
observed_difference = grouped.get('inspection_score').loc['Low Risk'] - grouped.get('inspection_score').loc['High Risk']
observed_difference

Then, for our shuffled data:

In [ ]:
original_and_shuffled.groupby('shuffled_label').mean()

In [ ]:
shuffled_and_grouped = original_and_shuffled.groupby('shuffled_label').mean()
simulated_difference = shuffled_and_grouped.get('inspection_score').loc['Low Risk'] - shuffled_and_grouped.get('inspection_score').loc['High Risk']
simulated_difference

We're going to need to shuffle the `'risk_category'` column many, many times, and compute this difference in group means each time.

Let's put some of our code in a function to make it easier to repeat.

In [ ]:
def calculate_test_statistic():
    shuffled_labels = np.random.permutation(high_low.get('risk_category'))
    original_and_shuffled = high_low.assign(shuffled_label=shuffled_labels)
    shuffled_and_grouped = original_and_shuffled.groupby('shuffled_label').mean()
    simulated_difference = shuffled_and_grouped.get('inspection_score').loc['Low Risk'] - shuffled_and_grouped.get('inspection_score').loc['High Risk']
    return simulated_difference

Each time we call this function, it shuffles the `'risk_category'` column and returns the difference in group means (again, by taking low minus high).

In [ ]:
calculate_test_statistic()

We need to simulate this difference in group means many, many times. Let's call our function many, many times and keep track of its result in an array.

In [ ]:
simulated_stats = np.array([])

n_reps = 100 # We're using a small number of reps to keep the runtime low

for i in np.arange(n_reps):
    sim_stat = calculate_test_statistic()
    simulated_stats = np.append(simulated_stats, sim_stat)

Now that we've done that, let's visualize the distribution of the simulated test statistics, and also see where the observed statistic lies:

In [ ]:
bpd.DataFrame().assign(simulated_stats=simulated_stats) \
               .plot(kind='hist', density=True, ec='w', figsize=(10, 5), bins=20, label='difference in group means');
plt.axvline(observed_difference, color='red', label='observed statistic')
plt.legend();

What's the p-value? Well, it depends on what our alternative hypothesis is.

Here, our alternative hypothesis is that low risk restaurants have higher inspection scores on average than high risk restaurants. Since our test statistic was calculated by taking the low mean minus the high mean, larger values of the test statistic favor the alternative.

In [ ]:
np.count_nonzero(simulated_stats >= observed_difference) / n_reps

This is lower than any cutoff we'd consider, so we'd reject the null hypothesis that the two groups of restaurants have similar inspection scores.

## Bakeries 🧁

<center><img src='data/cupcake.png' width=600></center>
<center>by Chef Janine</center>

### The Central Limit Theorem

> The Central Limit Theorem (CLT) says that the probability distribution of the **sum or average** of a large random sample drawn with replacement will be roughly normal, regardless of the distribution of the population from which the sample is drawn.

In [ ]:
show_clt_slides()

We'll load in a version of the restaurants dataset that has many more rows, some of which contain null values.

In [ ]:
restaurants_full = bpd.read_csv('data/restaurants.csv').get(keep_cols)
restaurants_full

Let's look at just the restaurants with `'Bake'` in the name that we know the inspection score for.

`.str.contains` can help us here.

In [ ]:
restaurants_full.get('business_name').str.contains('Bake')

Some bakeries may have `'bake'` in their name, rather than `'Bake'`. To account for this, we can convert the entire Series to lowercase using `.str.lower()`, and then use `.str.contains('bake')`.

In [ ]:
restaurants_full.get('business_name').str.lower().str.contains('bake')

In [ ]:
bakeries = restaurants_full[restaurants_full.get('business_name').str.lower().str.contains('bake')]
bakeries = bakeries[bakeries.get('inspection_score') >= 0] # Keeping only the rows where we know the inspection score
bakeries

We can plot the **population** distribution, i.e. the distribution of inspection scores for **all bakeries in San Francisco**.

In [ ]:
bakeries.plot(kind='hist', y='inspection_score', density=True, bins=score_bins, ec='w', figsize=(10, 5),
              title='Population Distribution');

For reference, the mean and standard deviation of the population distribution are calculated below.

In [ ]:
bakeries.get('inspection_score').mean()

In [ ]:
np.std(bakeries.get('inspection_score'))

In this case we happen to have the inspection scores for all members of the population, but in reality we won't. So let's instead consider a random **sample** of the population.

In [ ]:
np.random.seed(23) # Ignore this

sample_of_bakeries = bakeries.sample(200, replace=False)
sample_of_bakeries

We can plot the sample distribution:

In [ ]:
sample_of_bakeries.plot(kind='hist', y='inspection_score', density=True, bins=score_bins, ec='w', figsize=(10, 5),
                        title='Sample Distribution');

Note that since we took a large, random sample of the population, we expect that our sample looks similiar to the population and has a similar mean and SD.

In [ ]:
sample_of_bakeries.get('inspection_score').mean()

In [ ]:
np.std(sample_of_bakeries.get('inspection_score'))

Indeed, the sample mean is quite close to the population mean, and the sample standard deviation is quite close to the population standard deviation.

Let's suppose we want to estimate the population mean (that is, the mean inspection score of all bakeries in SF).

One estimate of the population mean is the mean of our sample.

In [ ]:
sample_of_bakeries.get('inspection_score').mean()

However, our sample was random and could have been different, meaning our sample mean could also have been different.

**Question:** What's a reasonable range of possible values for the sample mean? **What is the distribution of the sample mean?**

**The Central Limit Theorem tells us what the distribution of the sample mean is.** To see the distribution of the sample mean visually, let's take a large number of samples directly from the population and compute the mean of each one.

In [ ]:
sample_means = np.array([])

for i in np.arange(5000):
    sample_mean = bakeries.sample(200, replace=False).get('inspection_score').mean()
    sample_means = np.append(sample_means, sample_mean)

In [ ]:
sample_means

In [ ]:
bpd.DataFrame().assign(sample_means=sample_means).plot(kind='hist', density=True, ec='w', bins=25, figsize=(10, 5));

Unsurprisingly, the distribution of the sample mean is bell-shaped. The CLT told us that!

The CLT also tells us that

$$\text{SD of Distribution of Possible Sample Means} = \frac{\text{Population SD}}{\sqrt{\text{sample size}}}$$



Let's try this out.

In [ ]:
np.std(bakeries.get('inspection_score')) / np.sqrt(200)

In [ ]:
np.std(sample_means)

Pretty close! Remember that `sample_means` is an array of simulated sample means; the more samples we simulate, the closer that `np.std(sample_means)` will get to the SD described by the CLT.

Note that in practice, we won't have the SD of the population, since we'll usually just have a single sample. In such cases, we can use the SD of the sample as an estimate of the SD of the population:

In [ ]:
np.std(sample_of_bakeries.get('inspection_score')) / np.sqrt(200)

Using the CLT, we have that the distribution of the sample mean:
- is roughly normal,
- is centered at the population mean (for which the sample mean is an estimate), and
- has a standard deviation of $\frac{\text{Population SD}}{\sqrt{\text{sample size}}}$ (which can be estimated using $\frac{\text{Sample SD}}{\sqrt{\text{sample size}}}$).

Using this information, we can build confidence intervals for where we think the population mean might be. For instance, a 95% confidence interval for the population mean is given by

$$
\left[
\text{sample mean} - 2\cdot \frac{\text{sample SD}}{\sqrt{n}}, \
\text{sample mean} + 2\cdot \frac{\text{sample SD}}{\sqrt{n}}
\right]
$$

In [ ]:
sample_mean = sample_of_bakeries.get('inspection_score').mean()
sample_std = np.std(sample_of_bakeries.get('inspection_score'))

In [ ]:
[sample_mean - 2 * sample_std / np.sqrt(200), sample_mean + 2 * sample_std / np.sqrt(200)]

### Discussion Question

Using a single sample of 200 bakeries, how can we estimate the **median** inspection score of all bakeries in San Francisco with an inspection score? What technique should we use?

A. Standard hypothesis testing

B. Permutation testing  

C. Bootstrapping

D. The Central Limit Theorem

### To answer, go to [menti.com](https://menti.com) and enter the code 4857 5944.

<details>
<summary>Click for the answer <b>after</b> you've entered your guess above. <b>Don't scroll any further.</b></summary>
    
Bootstrapping. The CLT only applies to sample means (and sums), not to any other statistics.

</details>

There is no CLT for sample medians, so instead we'll have to resort to bootstrapping to estimate the distribution of the sample median.

Recall, bootstrapping is the act of **sampling from the original sample, with replacement**. This is also called **resampling**.

In [ ]:
# The median of our original sample – this is just one number
sample_of_bakeries.get('inspection_score').median()

In [ ]:
# The median of a single bootstrap resample – this is just one number
sample_of_bakeries.sample(200, replace=True).get('inspection_score').median()

Let's resample repeatedly.

In [ ]:
np.random.seed(23) # Ignore this

boot_medians = np.array([])

for i in np.arange(5000):
    boot_median = sample_of_bakeries.sample(200, replace=True).get('inspection_score').median()
    boot_medians = np.append(boot_medians, boot_median)

In [ ]:
boot_medians

In [ ]:
bpd.DataFrame().assign(boot_medians=boot_medians).plot(kind='hist', density=True, ec='w', bins=10, figsize=(10, 5));

Note that this distribution is not at all normal.

To compute a 95% confidence interval, we take the middle 95% of the bootstrapped medians.

In [ ]:
left = np.percentile(boot_medians, 2.5)
right = np.percentile(boot_medians, 97.5)

[left, right]

### Discussion Question

Which of the following interpretations of this confidence interval are valid?

1. 95% of SF bakeries have an inspection score between 85 and 88.  
2. 95% of the resamples have a median inspection score between 85 and 88.  
3. There is a 95% chance that our sample has a median inspection score between 85 and 88.  
4. There is a 95% chance that the median inspection score of all SF bakeries is between 85 and 88.  
5. If we had taken 100 samples from the same population, about 95 of these samples would have a median inspection score between 85 and 88.  
6.  If we had taken 100 samples from the same population, about 95 of the confidence intervals created would contain the median inspection score of all SF bakeries.  

<details>
<summary>Click for the answer <b>after</b> you've entered your guess above. <b>Don't scroll any further.</b></summary>
    
The correct answers are Option 2 and Option 6.

</details>

## Physicians 🩺

### The setup

You work as a family physician. You collect data and you find that in 6354 patients, 3115 were children and 3239 were adults.

You want to test the following hypotheses:

- **Null Hypothesis:** Family physicians see an equal number of children and adults.
- **Alternative Hypothesis:** Family physicians see more adults than they see children.

### Discussion Question

Which test statistic(s) could be used for this hypothesis test? Which values of the test statistic point towards the alternative?

A. proportion of children seen   
B. number of children seen  
C. number of children minus number of adults seen  
D. absolute value of number of children minus number of adults seen

**There may be multiple correct answers.**

<details>
<summary>Click for the answer <b>after</b> you've entered your guess above. <b>Don't scroll any further.</b></summary>
    
All of these but the last one would work for this alternative. Small values of these statistics would favor the alternative.
    
If the alternative was instead "Family physicians see a different number of children and adults", the last option would work while the first three wouldn't.

</details>

Let's use option B, the number of children seen, as a test statistic. Small values of this statistic favor the alternative hypothesis.

How do we generate a single value of the test statistic?

In [ ]:
np.random.multinomial(6354, [0.5, 0.5])[0]

As per usual, let's simulate the test statistic many, many times.

In [ ]:
test_stats = np.array([])

for i in np.arange(10000):
    stat = np.random.multinomial(6354, [0.5, 0.5])[0]
    test_stats = np.append(test_stats, stat)

In [ ]:
test_stats

In [ ]:
bpd.DataFrame().assign(test_stats=test_stats) \
               .plot(kind='hist', density=True, ec='w', figsize=(10, 5), bins=20);
plt.axvline(3115, color='red', label='observed statistic')
plt.legend();

Recall that you collected data and found that in 6354 patients, 3115 were children and 3239 were adults.

### Discussion Question

What goes in blank (a)?

```py
p_value = np.count_nonzero(test_stats __(a)__ 3115) / 10000
```

A. `>=`

B. `>`

C. `<=`

D. `<`

### To answer, go to [menti.com](https://menti.com) and enter the code 4857 5944.

<details>
<summary>Click for the answer <b>after</b> you've entered your guess above. <b>Don't scroll any further.</b></summary>
    <=

</details>

In [ ]:
# Calculate the p-value

### Discussion Question

What do we do, assuming that we're using a 5% p-value cutoff?

A. reject the null  

B. fail to reject the null 

C. not sure

### To answer, go to [menti.com](https://menti.com) and enter the code 4857 5944.

<details>
<summary>Click for the answer <b>after</b> you've entered your guess above. <b>Don't scroll any further.</b></summary>
    Fail to reject the null, since the p-value is above 0.05.

</details>

Note that while we used `np.random.permutation` to simulate the test statistic, we could have used `np.random.choice`, too:

In [ ]:
choices = np.random.choice(['adult', 'child'], p=[0.5, 0.5], size=6354, replace=True)
choices

In [ ]:
np.count_nonzero(choices == 'child')

### Discussion Question

Is this an example of bootstrapping?

A. Yes, because we are sampling with replacement.

B. No, this is not bootstrapping.

### To answer, go to [menti.com](https://menti.com) and enter the code 4857 5944.

<details>
<summary>Click for the answer <b>after</b> you've entered your guess above. <b>Don't scroll any further.</b></summary>
    No, this is not bootstrapping. Bootstrapping is when we resample from a single sample; here we're simulating data under the assumptions of a model.

</details>

## Summary

### Next time

- Class on Friday will be a high-level overview of the quarter + a few cool demos on things you'll learn in later DSC courses.